In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

x=pd.read_csv('Eazy_Dinner.csv')
x.head()

,Unnamed: 0,name,rating,cuisine,location,region,cost_per_person,discount
0,0,Chin Lung Resto Bar,4.6,Multicuisine,['Residency Road'],['Central Bengaluru'],2000,25
1,1,Chin Lung Resto Bar,4.6,Multicuisine,['Residency Road'],['Central Bengaluru'],2000,25
2,2,"Atlantis - Brewpub, Cocktails & Kitchen",4.2,Multicuisine,['HSR'],['South Bengaluru'],2000,25
3,3,"Atlantis - Brewpub, Cocktails & Kitchen",4.2,Multicuisine,['HSR'],['South Bengaluru'],2000,25
4,4,Chin Lung Brewery - Indiranagar,4.6,Multicuisine,['Indiranagar'],['East Bengaluru'],2000,25


In [3]:
# STEP 1: BASIC INFO - EXAMINE DATASET STRUCTURE
print('\n' + '='*80)
print('STEP 1: DATASET OVERVIEW')
print('='*80)
print(f'\nDataset Shape: {x.shape[0]} rows, {x.shape[1]} columns')
print(f'\nColumn Names and Data Types:')
print(x.dtypes)
print(f'\nMissing Values:')
print(x.isna().sum())
print(f'\nFirst 5 rows:')
print(x.head())


STEP 1: DATASET OVERVIEW

Dataset Shape: 1600 rows, 8 columns

Column Names and Data Types:
Unnamed: 0           int64
name                object
rating             float64
cuisine             object
location            object
region              object
cost_per_person      int64
discount             int64
dtype: object

Missing Values:
Unnamed: 0         0
name               0
rating             0
cuisine            0
location           0
region             0
cost_per_person    0
discount           0
dtype: int64

First 5 rows:
   Unnamed: 0                                     name  rating       cuisine  \
0           0                      Chin Lung Resto Bar     4.6  Multicuisine   
1           1                      Chin Lung Resto Bar     4.6  Multicuisine   
2           2  Atlantis - Brewpub, Cocktails & Kitchen     4.2  Multicuisine   
3           3  Atlantis - Brewpub, Cocktails & Kitchen     4.2  Multicuisine   
4           4          Chin Lung Brewery - Indiranagar     4.6  